In [1]:
import pandas as pd
import numpy as np
import igraph as ig
import matplotlib.pyplot as plt
from subprocess import call
import sys
sys.path.append('../')
from package_global_functions import *
from filesHandling_kilombo import *

latexFont(ticklabelssize=12,labelsize=14) # ticklabelssize=10, labelsize=12, legendsize=8

In [21]:
def dist2D(pos0, pos1):
    return np.sqrt((pos0[0]-pos1[0])**2+(pos0[1]-pos1[1])**2)

def getDistances(positions, i, cicle, interac_r):
    dists = []
    for k,pos0 in enumerate(positions[:-1]):
        for pos1 in positions[k+1:]:
            dists.append(dist2D(pos0, pos1))
    dists = np.array(dists)
    dists = dists[dists < 40.0]
    figD, axD = plt.subplots()
    axD.hist(dists)
    axD.axvline(interac_r, ls='--', color='k')
    figD.tight_layout()
    figD.savefig(f'histoDists_config_{str(i).zfill(3)}_cicle_{cicle}.png')
    dists = dists[dists <= interac_r]
    print(f'{len(dists)} distances smaller thant interac_r')

In [20]:
arena_r = 185.0
exclusion_r = 15.0
interac_r = 55.0
cicle = 10
loops = 800

N = 25
jumpTrajConfigs = 5

# parquet files:
filenameConfig = getConfigsPath() + '/' + getFilenameRoot(N, arena_r/10) + getFilenameNumber(1) + getFilesExtension()
filenameContact = getConfigsPath() + '/contacts/' + getFilenameRoot(N, arena_r/10) + getFilenameNumber(1) + getFilenameContactSufix(loops, interac_r/10)
dfconfigs = pd.read_parquet(filenameConfig)
dfcontacts = pd.read_parquet(filenameContact)

ids = pd.unique(dfconfigs['ID'])
Nbots = len(ids) # should be == N
dfcicle0 = dfcontacts.loc[dfcontacts['cicleID']==cicle]
configs0 = pd.unique(dfcicle0['configID'])
print(configs0)

[153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168]


In [8]:
# format = 'pdf'
format = 'png'
call('mkdir -p figures_for_video', shell=True)
call(f'rm figures_for_video/*.png', shell=True)
call(f'rm figures_for_video/*.pdf', shell=True)
for i in configs0:
    df_single_config_contacts = dfcicle0.loc[dfcicle0['configID']==i].copy(deep=True)
    df_single_config_contacts.drop(labels=['cicleID', 'configID'], axis='columns', inplace=True)
    g = ig.Graph.DataFrame(df_single_config_contacts, directed=False)
    imod = i*jumpTrajConfigs
    df_single_config = dfconfigs.loc[(imod*Nbots):(imod*Nbots+Nbots-1)]
    if len(g.vs) < Nbots:
        lacking_bots = [k for k in range(len(g.vs),Nbots)]
        for k in lacking_bots:
            g.add_vertex() # name=v not necessary as the others don't have name anyway
    true_layout = []
    for j in range(Nbots):
        x, y = float(df_single_config.loc[df_single_config['ID']==j]['x_position'].iloc[0]), float(df_single_config.loc[df_single_config['ID']==j]['y_position'].iloc[0])
        true_layout.append([x,y])
    # getDistances(true_layout, i, cicle, interac_r)
    true_layout = ig.Layout(true_layout)
    fig, ax = plt.subplots(figsize=(5,5), constrained_layout=True)
    plt.axis('off')
    ig.plot(
        g, target=ax, layout=true_layout, vertex_size = 0.0, vertex_color = 'white', vertex_label=[j for j in range(Nbots)],
        vertex_frame_width = 0.0, vertex_frame_color = 'white', edge_width=2.0, edge_color='black'
    )
    circle = plt.Circle([0.0, 0.0], arena_r+exclusion_r, fill=False, edgecolor = 'k')
    ax.add_patch(circle)
    ax.set_xlim(-(arena_r+exclusion_r+1), arena_r+exclusion_r+1)
    ax.set_ylim(-(arena_r+exclusion_r+1), arena_r+exclusion_r+1)
    # fig.suptitle(f'config {i} in cycle {cicle}')
    for j in range(Nbots):
        coords = true_layout[j]
        circle = plt.Circle(tuple(coords), exclusion_r, color='xkcd:purple', alpha=0.5, clip_on = False)
        ax.add_patch(circle)
    # extra: show interaction radius for the ID 0 bot
    id0pos = (float(df_single_config.query('ID == 0')['x_position'].iloc[0]), float(df_single_config.query('ID == 0')['y_position'].iloc[0]))
    circle = plt.Circle(id0pos, interac_r, fill=False, edgecolor='xkcd:grey', ls='--')
    ax.add_patch(circle)
    fig.savefig(f'config_{str(i).zfill(3)}_cicle_{cicle}.{format}', dpi=300)
    plt.close(fig)
    call(f'mv config_{str(i).zfill(3)}_cicle_{cicle}.{format} figures_for_video/', shell=True)

rm: figures_for_video/*.pdf: No such file or directory


In [14]:
np.arange(0,1,0.2)

array([0. , 0.2, 0.4, 0.6, 0.8])

## more configs

In [25]:
jumpTrajConfigs = 5
configs0_more = []
configs0_more_aux = []
for c in configs0:
    configs0_more.extend(list(np.around(np.arange(c,c+1,0.2),1)))
    configs0_more_aux.extend([c]*jumpTrajConfigs)
print(configs0_more)
# configs0_more = []
jumpTrajConfigs_new = 1

# format = 'pdf'
format = 'png'
call('mkdir -p figures_for_video', shell=True)
call(f'rm figures_for_video/*.png', shell=True)
call(f'rm figures_for_video/*.pdf', shell=True)
for i,(c,caux) in enumerate(zip(configs0_more,configs0_more_aux)):
    df_single_config_contacts = dfcicle0.loc[dfcicle0['configID']==caux].copy(deep=True)
    df_single_config_contacts.drop(labels=['cicleID', 'configID'], axis='columns', inplace=True)
    g = ig.Graph.DataFrame(df_single_config_contacts, directed=False)
    imod = int(c*jumpTrajConfigs)
    df_single_config = dfconfigs.loc[(imod*Nbots):(imod*Nbots+Nbots-1)]
    if len(g.vs) < Nbots:
        lacking_bots = [k for k in range(len(g.vs),Nbots)]
        for k in lacking_bots:
            g.add_vertex() # name=v not necessary as the others don't have name anyway
    true_layout = []
    for j in range(Nbots):
        x, y = float(df_single_config.loc[df_single_config['ID']==j]['x_position'].iloc[0]), float(df_single_config.loc[df_single_config['ID']==j]['y_position'].iloc[0])
        true_layout.append([x,y])
    # getDistances(true_layout, i, cicle, interac_r)
    true_layout = ig.Layout(true_layout)
    fig, ax = plt.subplots(figsize=(5,5), constrained_layout=True)
    plt.axis('off')
    ig.plot(
        g, target=ax, layout=true_layout, vertex_size = 0.0, vertex_color = 'white', vertex_label=[j for j in range(Nbots)],
        vertex_frame_width = 0.0, vertex_frame_color = 'white', edge_width=2.0, edge_color='gray'
    )
    circle = plt.Circle([0.0, 0.0], arena_r+exclusion_r, fill=False, edgecolor = 'k')
    ax.add_patch(circle)
    ax.set_xlim(-(arena_r+exclusion_r+1), arena_r+exclusion_r+1)
    ax.set_ylim(-(arena_r+exclusion_r+1), arena_r+exclusion_r+1)
    # fig.suptitle(f'config {i} in cycle {cicle}')
    for j in range(Nbots):
        coords = true_layout[j]
        circle = plt.Circle(tuple(coords), exclusion_r, color='xkcd:purple', alpha=0.5, clip_on = False)
        ax.add_patch(circle)
    # extra: show interaction radius for the ID 0 bot
    id0pos = (float(df_single_config.query('ID == 0')['x_position'].iloc[0]), float(df_single_config.query('ID == 0')['y_position'].iloc[0]))
    circle = plt.Circle(id0pos, interac_r, fill=False, edgecolor='xkcd:grey', ls='--')
    ax.add_patch(circle)
    fig.savefig(f'config_{str(i).zfill(3)}_cicle_{cicle}.{format}', dpi=300)
    plt.close(fig)
    call(f'mv config_{str(i).zfill(3)}_cicle_{cicle}.{format} figures_for_video/', shell=True)

[153.0, 153.2, 153.4, 153.6, 153.8, 154.0, 154.2, 154.4, 154.6, 154.8, 155.0, 155.2, 155.4, 155.6, 155.8, 156.0, 156.2, 156.4, 156.6, 156.8, 157.0, 157.2, 157.4, 157.6, 157.8, 158.0, 158.2, 158.4, 158.6, 158.8, 159.0, 159.2, 159.4, 159.6, 159.8, 160.0, 160.2, 160.4, 160.6, 160.8, 161.0, 161.2, 161.4, 161.6, 161.8, 162.0, 162.2, 162.4, 162.6, 162.8, 163.0, 163.2, 163.4, 163.6, 163.8, 164.0, 164.2, 164.4, 164.6, 164.8, 165.0, 165.2, 165.4, 165.6, 165.8, 166.0, 166.2, 166.4, 166.6, 166.8, 167.0, 167.2, 167.4, 167.6, 167.8, 168.0, 168.2, 168.4, 168.6, 168.8]


rm: figures_for_video/*.pdf: No such file or directory


In [26]:
## VIDEO
# many options to do so: https://stackoverflow.com/questions/44947505/how-to-make-a-movie-out-of-images-in-python
import os
import glob
import moviepy.video.io.ImageSequenceClip
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

image_files = []
path_to_images = 'figures_for_video'

image_files = sorted(glob.glob(f'{path_to_images}/*.png'))

fps = 10

clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=fps)
clip.write_videofile(f'N_{N}_ir_{interac_r/10}_cicle_{cicle}.mp4')

Moviepy - Building video N_25_ir_5.5_cicle_10.mp4.
Moviepy - Writing video N_25_ir_5.5_cicle_10.mp4



Moviepy - Done !
Moviepy - video ready N_25_ir_5.5_cicle_10.mp4


In [5]:
# print the corresponding integrated config:
filenameContactInt = getConfigsPath() + '/contacts/' + getFilenameRoot(N, arena_r/10) + getFilenameNumber(1) + getFilenameContactIntSufix(loops, interac_r/10)
dfcontactsINT = pd.read_parquet(filenameContactInt)
dfcontactsINTcicle0 = dfcontactsINT.loc[dfcontactsINT['cicleID']==cicle].copy(deep=True)
dfcontactsINTcicle0.drop(labels='cicleID', axis='columns', inplace=True)
# dfcontactsINTcicle0.sort_values(by=['contacts0', 'contacts1'], inplace=True)
# dfcontactsINTcicle0.reset_index(drop=True, inplace=True)
g = ig.Graph.DataFrame(dfcontactsINTcicle0, directed=False)
if len(g.vs) < Nbots:
    lacking_bots = [k for k in range(len(g.vs),Nbots)]
    for k in lacking_bots:
        g.add_vertex() # name=v not necessary as the others don't have name anyway
fig, ax = plt.subplots(figsize=(5,5), constrained_layout=True)
plt.axis('off')
# use as layout the last positions in the cycle
i = configs0[-1]
imod = i*jumpTrajConfigs
df_single_config = dfconfigs.loc[(imod*Nbots):(imod*Nbots+Nbots-1)]
true_layout = []
for j in range(N):
    x, y = float(df_single_config.loc[df_single_config['ID']==j]['x_position'].iloc[0]), float(df_single_config.loc[df_single_config['ID']==j]['y_position'].iloc[0])
    true_layout.append([x,y])
true_layout = ig.Layout(true_layout)
ig.plot(
    g, target=ax, layout=true_layout, vertex_size = 0.0, vertex_color = 'white', vertex_label=[j for j in range(Nbots)], 
    vertex_frame_width = 0.0, vertex_frame_color = 'white', edge_width=2.0, edge_color='black'
)
circle = plt.Circle([0.0, 0.0], arena_r+exclusion_r, fill=False, edgecolor = 'k')
ax.add_patch(circle)
ax.set_xlim(-(arena_r+exclusion_r+1), arena_r+exclusion_r+1)
ax.set_ylim(-(arena_r+exclusion_r+1), arena_r+exclusion_r+1)
for j in range(N):
    coords = true_layout[j]
    circle = plt.Circle(tuple(coords), exclusion_r, color='xkcd:tan', alpha=0.5, clip_on = False)
    ax.add_patch(circle)
    # ax.text(coords[0], coords[1], f'{v}')
fig.savefig(f'N_{N}_ir_{interac_r/10}_cicle_{cicle}_integrated_config.png', dpi=300)
plt.close(fig)